# Optimizing the hyperparameters for running a simple fully connected neural network model

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import required libraries
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# LabelEncoder is important for neural network analysis for converting output labels array
# into one hot encoded matrix of K columns where K is the unqiue number of categories
%matplotlib inline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [0]:
# read in data
# dataset_redwine = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L2/data_rnn_model/redwine_data.csv', sep = ';')
dataset_redwine = pd.read_csv('redwine_fully_NN.csv', sep = ';')
dataset_redwine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [0]:
# data pre-processing
input_features = dataset_redwine.iloc[:,0:11].values
targets = dataset_redwine.iloc[:,11].values
encoder = LabelEncoder()
y_transformed = encoder.fit_transform(targets) # convert the labels into a series of integers
exact_labels = pd.get_dummies(y_transformed).values # convert the series of integers into a encoded matrix

# data scaling for the input features
features_scaler = StandardScaler()
scaled_train_data = features_scaler.fit_transform(input_features)

In [0]:
# setting up fully connected NN model within a defined classifier function
def train_classifer():
  model_ANN = Sequential() # initialize the model
  # create the input layer 
  input_layer = Dense(units = 9, kernel_initializer = 'uniform',
                      activation = 'relu', input_dim = 11)
  model_ANN.add(input_layer)
  
  # setting up 3 hidden layers
  hidden_layer1 = Dense(units = 9,kernel_initializer = 'uniform',
                      activation = 'relu')
  hidden_layer2 = Dense(units = 7,kernel_initializer = 'uniform',
                      activation = 'relu')
  hidden_layer3 = Dense(units = 7,kernel_initializer = 'uniform',
                      activation = 'relu')
  
  # create output layer 
  output_layer = Dense(units = 6, kernel_initializer = 'uniform',
                      activation = 'softmax')
  model_ANN.add(hidden_layer1)
  model_ANN.add(hidden_layer2)
  model_ANN.add(hidden_layer3)
  model_ANN.add(output_layer)
  
  # compile model 
  model_ANN.compile(optimizer = 'adam',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])
  return model_ANN

In [0]:
# implementing cross_validation
ann_clf = KerasClassifier(build_fn = train_classifer,
                          batch_size = 16,
                          epochs = 100)

results_cross = cross_val_score(estimator = ann_clf,
                                X = scaled_train_data,
                                y = exact_labels,
                                cv = 3)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1066/1066 [==============================] - 3s 3ms/step - loss: 1.7557 - acc: 0.3799
Epoch 2/100
1066/1066 [==============================] - 0s 383us/step - loss: 1.5505 - acc: 0.3987
Epoch 3/100
1066/1066 [==============================] - 0s 365us/step - loss: 1.2047 - acc: 0.4034
Epoch 4/100
1066/1066 [==============================] - 0s 364us/step - loss: 1.1623 - acc: 0.4034
Epoch 5/100
1066/1066 [==============================] - 0s 368us/step - loss: 1.1468 - acc: 0.4034
Epoch 6/100
1066/1066 [==============================] - 0s 385us/step - loss: 1.1358 - acc: 0.4053
Epoch 7/100
1066/1066 [==============================] - 0s 377us/step - loss: 1.1274 - acc: 0.4325
Epoch 8/100
1066/1066 [==============================] - 0s 377us/step - loss: 1.1183 - acc: 0.4719
Epoch 9/100
1066/1066 [==============================] - 0s 377us/step - loss: 1.1

In [0]:
print(results_cross)
print(results_cross.mean())
print(results_cross.std())

[0.57410882 0.60412758 0.57598499]
0.5847404628824338
0.01373014285376921


In [0]:
grid_parameters = {'batch_size':[16,32], 'epochs':[300,500]}
ann_clf = KerasClassifier(build_fn = train_classifer)
GRID_SEARCH = GridSearchCV(estimator = ann_clf,
                           param_grid = grid_parameters,
                           scoring = 'accuracy',
                           cv = 3)
GRID_SEARCH_RESULTS = GRID_SEARCH.fit(scaled_train_data, y_transformed)

Epoch 1/300
1066/1066 [==============================] - 1s 968us/step - loss: 1.7584 - acc: 0.4081
Epoch 2/300
1066/1066 [==============================] - 0s 448us/step - loss: 1.6156 - acc: 0.4156
Epoch 3/300
1066/1066 [==============================] - 0s 404us/step - loss: 1.3041 - acc: 0.4156
Epoch 4/300
1066/1066 [==============================] - 0s 397us/step - loss: 1.2150 - acc: 0.3987
Epoch 5/300
1066/1066 [==============================] - 0s 396us/step - loss: 1.1802 - acc: 0.4034
Epoch 6/300
1066/1066 [==============================] - 0s 410us/step - loss: 1.1625 - acc: 0.4128
Epoch 7/300
1066/1066 [==============================] - 0s 384us/step - loss: 1.1506 - acc: 0.4034
Epoch 8/300
1066/1066 [==============================] - 0s 400us/step - loss: 1.1410 - acc: 0.4212
Epoch 9/300
1066/1066 [==============================] - 0s 388us/step - loss: 1.1330 - acc: 0.4053
Epoch 10/300
1066/1066 [==============================] - 0s 378us/step - loss: 1.1266 - acc: 0.4690

In [0]:
# printing out the optimal set of selected group of hyperparameters (batch_size, epochs)
para_results = GRID_SEARCH_RESULTS.best_params_
print('The optimal batch size and number of epochs for the running the NN model are ' \
     + str(para_results['batch_size'] + ' and ' + str(para_results['epochs'] + 'respectively.')))